In [ ]:
##궁극의 코드!!! 절대 지우면 안됨!!###


import os, time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoAlertPresentException

DOWNLOAD_DIR = r"C:\Users\mstel\Downloads"
options = Options()
options.add_experimental_option("detach", True)

driver = webdriver.Chrome(options=options)
driver.get("https://tmacs.kotsa.or.kr/web/TG/TG300/TG3100/Tg2127.jsp?mid=S1810")
wait = WebDriverWait(driver, 10)

# ---------------------- 팝업 / 다운로드 ----------------------
def wait_for_download_complete(download_dir=DOWNLOAD_DIR, timeout=30):
    before = set(os.listdir(download_dir))
    end_time = time.time() + timeout
    while time.time() < end_time:
        after = set(os.listdir(download_dir))
        new_files = after - before
        if new_files and any(f.endswith(".xlsx") or f.endswith(".xls") for f in new_files):
            return list(new_files)[0]
        time.sleep(1)
    return None

def handle_structure_download(driver, wait, location_name):
    main_window = driver.current_window_handle
    retries = 0
    while retries < 2:
        try:
            time.sleep(1) # 원래 코드에는 있었던 sleep 유지
            for handle in driver.window_handles:
                if handle != main_window:
                    driver.switch_to.window(handle)
                    break

            tab = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#tabId03")))
            tab.click()
            time.sleep(4)

            wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "dl.jogun dd")))
            dd_elements = driver.find_elements(By.CSS_SELECTOR, "dl.jogun dd")

            사고지역 = dd_elements[0].text.strip() if len(dd_elements) > 0 else ""
            사고장소 = dd_elements[1].text.strip() if len(dd_elements) > 1 else ""
            조회기간 = dd_elements[2].text.strip() if len(dd_elements) > 2 else ""

            time.sleep(3)

            excel_btn = wait.until(EC.element_to_be_clickable(
                (By.CSS_SELECTOR, "#new_popup > div.pop_cont > div.btn_box > a.btn.exbtn")
            ))
            excel_btn.click()

            downloaded_file = wait_for_download_complete()
            if downloaded_file:
                downloaded_path = os.path.join(DOWNLOAD_DIR, downloaded_file)

                df = pd.read_excel(downloaded_path, header=[0, 1, 2])

                if isinstance(df.columns, pd.MultiIndex):
                    df.columns = ['_'.join([str(c) for c in col if str(c) != 'nan']) for col in df.columns]
                else:
                    df.columns = [str(c) for c in df.columns]

                df.insert(0, '사고지역', 사고지역)
                df.insert(1, '사고장소', 사고장소)
                df.insert(2, '조회기간', 조회기간)

                base_filename = "사고분석-지역별.xlsx"
                save_path = os.path.join(DOWNLOAD_DIR, base_filename)
                if os.path.exists(save_path):
                    i = 1
                    while True:
                        numbered_path = os.path.join(DOWNLOAD_DIR, f"사고분석-지역별 ({i}).xlsx")
                        if not os.path.exists(numbered_path):
                            save_path = numbered_path
                            break
                        i += 1

                df.to_excel(save_path, index=False)
                print(f"[✅ 성공] {location_name} → {save_path}")
                os.remove(downloaded_path)
                break
            else:
                print(f"[⚠️ 타임아웃] {location_name}")

        except Exception as e:
            retries += 1
            print(f"[❌ 실패] {location_name} - {e} (재시도 {retries}/2)")
            driver.save_screenshot(f"error_{location_name}.png")
            time.sleep(1)

        finally:
            if len(driver.window_handles) > 1:
                driver.close()
                driver.switch_to.window(main_window)

    if retries == 2:
        print(f"[⛔ 완전 실패] {location_name}")

# ---------------------- visited ----------------------
visited = set()

# ---------------------- 버튼 클릭 + 내부 스크롤 ----------------------
def download_page(driver, wait, year_value, sido_text, jijace_text, page_number):
    scroll_element = driver.find_element(By.CSS_SELECTOR, ".rMateH5__VBrowserScrollBar")

    while True:
        buttons = driver.find_elements(By.CSS_SELECTOR, "#rMateH5__Content69 > div > span > img")
        if not buttons:
            print("버튼 없음 → 종료")
            break

        for idx, btn in enumerate(buttons):
            try:
                button_cell = btn.find_element(By.XPATH, "./ancestor::div[contains(@class, 'rMateH5__ImageItemRenderer')]")
                parent_container = button_cell.find_element(By.XPATH, "./parent::div")
                branch_name = parent_container.find_element(By.CSS_SELECTOR, "span.rMateH5__DataGridColumn25").text.strip()
                incident_count = parent_container.find_element(By.CSS_SELECTOR, "span.rMateH5__DataGridColumn27").text.strip()

                key = f"{branch_name}_{incident_count}_{idx}"
                if key in visited:
                    continue

                print(f"👉 클릭: {key}")
                driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", btn)
                time.sleep(0.5)
                driver.execute_script("arguments[0].click();", btn)
                time.sleep(1)

                handle_structure_download(driver, wait,
                                          f"{year_value}_{sido_text}_{jijace_text}_page{page_number}_{key}")
                visited.add(key)
                time.sleep(1)

            except Exception as e:
                print(f"[❌ 클릭 실패] {e}")
                driver.save_screenshot("click_error.png")
                time.sleep(1)

        # 스크롤 상태 체크
        current_scroll = driver.execute_script("return arguments[0].scrollTop;", scroll_element)
        max_scroll = driver.execute_script("return arguments[0].scrollHeight - arguments[0].clientHeight;", scroll_element)

        if current_scroll >= max_scroll:
            print("📌 스크롤 끝 → 종료")
            break
        elif current_scroll == current_scroll: # 이 부분은 원래 코드에 있었고, 기능상 의미 없지만 유지합니다.
             print("스크롤 위치 변화 없음. 추가 콘텐츠 로딩 대기 시도. (3초)")
             time.sleep(3)
        
        #last_scroll_height = current_scroll # 이 부분은 원래 코드에 없었으므로 포함하지 않습니다.

        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollTop + 500;", scroll_element)
        print("스크롤 다운...")
        time.sleep(1)

    print("✅ 이 페이지의 모든 버튼 다운로드 완료")

# ---------------------- 전체 다운로드 ----------------------
def download_all(driver, wait, year_value, sido_text, jijace_text):
    page = 1
    download_page(driver, wait, year_value, sido_text, jijace_text, page)
    print("✅ 전체 다운로드 완료")

# ---------------------- 실행 ----------------------
test_years = ["2023", "2022"]

for year_value in test_years:
    year_select = Select(driver.find_element(By.ID, "Year"))
    year_select.select_by_value(year_value)

    sido_options = driver.find_elements(By.CSS_SELECTOR, "#sido option")
    for sido_index in range(1, len(sido_options)):
        sido_select = Select(driver.find_element(By.ID, "sido"))
        sido_select.select_by_index(sido_index)
        sido_text = sido_select.first_selected_option.text
        
        wait.until(EC.presence_of_element_located((By.ID, "jijace")))
        jijace_options = driver.find_elements(By.CSS_SELECTOR, "#jijace option")
        for jijace_index in range(1, len(jijace_options)):
            jijace_select = Select(driver.find_element(By.ID, "jijace"))
            jijace_select.select_by_index(jijace_index)
            jijace_text = jijace_select.first_selected_option.text
            time.sleep(3) # 원래 코드에 있던 3초 대기 유지

            # --- 변경 시작: '조회' 버튼 클릭 방식 (ElementClickInterceptedException 해결) ---
            search_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div.btn_wrap > a")))
            driver.execute_script("arguments[0].click();", search_button)
            # --- 변경 끝 ---

            try:
                alert = driver.switch_to.alert
                alert.accept()
            except NoAlertPresentException:
                pass

            time.sleep(2) # 원래 코드에 있던 2초 대기 유지
            download_all(driver, wait, year_value, sido_text, jijace_text)

driver.quit()

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=138.0.7204.184)
Stacktrace:
	GetHandleVerifier [0x0x7ff67358e415+77285]
	GetHandleVerifier [0x0x7ff67358e470+77376]
	(No symbol) [0x0x7ff673359a6a]
	(No symbol) [0x0x7ff673331f21]
	(No symbol) [0x0x7ff6733df9be]
	(No symbol) [0x0x7ff673400192]
	(No symbol) [0x0x7ff6733d83e3]
	(No symbol) [0x0x7ff6733a1521]
	(No symbol) [0x0x7ff6733a22b3]
	GetHandleVerifier [0x0x7ff673871efd+3107021]
	GetHandleVerifier [0x0x7ff67386c29d+3083373]
	GetHandleVerifier [0x0x7ff67388bedd+3213485]
	GetHandleVerifier [0x0x7ff6735a884e+184862]
	GetHandleVerifier [0x0x7ff6735b055f+216879]
	GetHandleVerifier [0x0x7ff673597084+113236]
	GetHandleVerifier [0x0x7ff673597239+113673]
	GetHandleVerifier [0x0x7ff67357e298+11368]
	BaseThreadInitThunk [0x0x7ffb61411fe4+20]
	RtlUserThreadStart [0x0x7ffb6385ef91+33]
